# House Prices Competition : Term Project 

### Importing Libraries:

In [673]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn import neighbors
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

import matplotlib.pyplot as plt
plt.style.use(style='ggplot')
plt.rcParams['figure.figsize'] = (10, 6)

In [674]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
import lightgbm as lgb


In [675]:
# getting data from feature engineering notebook
%store -r train_set_1
%store -r test_set_1
%store -r ytrain_1

train_set = train_set_1
test_set = test_set_1

In [676]:
train_set.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,...,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Grvl,Street_Pave,SalePrice
0,856,854,0,0,3,706.0,0.0,1.0,0,150.0,...,0,0,0,0,0,0,1,0,1,208500
1,1262,0,0,0,3,978.0,0.0,0.0,1,284.0,...,0,0,0,0,0,0,1,0,1,181500
2,920,866,0,0,3,486.0,0.0,1.0,0,434.0,...,0,0,0,0,0,0,1,0,1,223500
3,961,756,0,0,3,216.0,0.0,1.0,0,540.0,...,0,0,0,0,0,0,1,0,1,140000
4,1145,1053,0,0,4,655.0,0.0,1.0,0,490.0,...,0,0,0,0,0,0,1,0,1,250000


In [677]:
print ("Train data shape:", train_set.shape)

Train data shape: (1452, 292)


In [678]:
test_set.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,...,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Grvl,Street_Pave
1452,896,0,0,0,2,468.0,144.0,0.0,0,270.0,...,0,0,0,0,0,0,0,1,0,1
1453,1329,0,0,0,3,923.0,0.0,0.0,0,406.0,...,0,0,0,0,0,0,0,1,0,1
1454,928,701,0,0,3,791.0,0.0,0.0,0,137.0,...,0,0,0,0,0,0,0,1,0,1
1455,926,678,0,0,3,602.0,0.0,0.0,0,324.0,...,0,0,0,0,0,0,0,1,0,1
1456,1280,0,0,0,2,263.0,0.0,0.0,0,1017.0,...,0,0,0,0,0,0,0,1,0,1


In [679]:
print ("Test data shape:", test_set.shape)


Test data shape: (1459, 291)


## Building the model:

In [680]:
train_set = train_set.select_dtypes(include=[np.number]).interpolate().dropna()
test_set = test_set.select_dtypes(include=[np.number]).interpolate().dropna()
train_set.head(5)

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,...,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Grvl,Street_Pave,SalePrice
0,856,854,0,0,3,706.0,0.0,1.0,0,150.0,...,0,0,0,0,0,0,1,0,1,208500
1,1262,0,0,0,3,978.0,0.0,0.0,1,284.0,...,0,0,0,0,0,0,1,0,1,181500
2,920,866,0,0,3,486.0,0.0,1.0,0,434.0,...,0,0,0,0,0,0,1,0,1,223500
3,961,756,0,0,3,216.0,0.0,1.0,0,540.0,...,0,0,0,0,0,0,1,0,1,140000
4,1145,1053,0,0,4,655.0,0.0,1.0,0,490.0,...,0,0,0,0,0,0,1,0,1,250000


In [681]:
from sklearn import preprocessing
#num_columns_train = train_set.select_dtypes(include = [np.number]).columns.values.tolist()
#num_columns_test = test_set.select_dtypes(include = [np.number]).columns.values.tolist()

#scaler = StandardScaler()

#train_set.loc[:,num_columns_train] = scaler.fit_transform(train_set[num_columns_train])
#test_set.loc[:,num_columns_test] = scaler.fit_transform(test_set[num_columns_test])


#print(train_set.head())
#print(test_set.head())

In [682]:
#y = train_set.SalePrice
y = np.log(train_set.SalePrice)
X = train_set.drop(['SalePrice'] , axis = 1)

### PCA:

In [683]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)                
t_train = scaler.transform(X)
pca_hp = PCA(30)
x_fit = pca_hp.fit_transform(t_train)
np.exp(pca_hp.explained_variance_ratio_)

array([ 1.0736579 ,  1.03167677,  1.02673244,  1.0238889 ,  1.02276893,
        1.01835754,  1.01627815,  1.01571762,  1.01227967,  1.01218986,
        1.01185609,  1.01121494,  1.01066472,  1.010115  ,  1.00984465,
        1.00965544,  1.00933163,  1.00929842,  1.00894703,  1.00862904,
        1.00842767,  1.00815987,  1.00809376,  1.00795234,  1.00777686,
        1.00769734,  1.00740914,  1.00727205,  1.00717364,  1.00714517])

train_test_split() returns four objects:

* X_train is the subset of our features used for training.
* X_test is the subset which will be our 'hold-out' set - what we'll use to test the model.
* y_train is the target variable SalePrice which corresponds to X_train.
* y_test is the target variable SalePrice which corresponds to X_test. 

random_state=42 allow reproducible results.

In [684]:
from sklearn.preprocessing import MinMaxScaler


X_train, X_test, y_train, y_test = train_test_split(
                                   X, y, random_state=42, test_size=.33)

from sklearn.preprocessing import StandardScaler
#scalerX = MinMaxScaler().fit(X_train)
#scalery = MinMaxScaler().fit(y_train.reshape(-1,1))
#X_train = scalerX.transform(X_train)
#y_train = scalery.transform(y_train.reshape(-1,1))
#X_test = scalerX.transform(X_test)
#y_test = scalery.transform(y_test.reshape(-1,1))


### Random Forest:

In [685]:
from sklearn.ensemble import RandomForestRegressor

# Fit Random Forest on Training Set
regressor = RandomForestRegressor(n_estimators=300, random_state=0)
model_random_forest = regressor.fit(X_train, y_train)


# Score model
print ("Test score is: ", model_random_forest.score(X_test, y_test))

Test score is:  0.875852404421


In [686]:
predictions_1 = model_random_forest.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_1)))

Mean squared Error : 0.019184894702


### KNN:

In [687]:
# find best k number

ourScore=[]
for nn in range(1,15):
    knn = neighbors.KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                                        metric_params=None, n_jobs=1, n_neighbors=nn,p=2,
                                        weights='uniform')
    knn.fit(X_train, y_train)
    train_score = knn.score(X_train, y_train)
    test_score = knn.score(X_test, y_test)
    #print('k: %d, Train Acc: %.3f, Test Acc: %.3f' % (nn, train_score, test_score))
    rowScore=[nn,train_score,test_score]
    ourScore.append(rowScore)

In [688]:
k=5
knn = neighbors.KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=k, p=2,
          weights='uniform')
knn.fit(X_train, y_train)
train_score = knn.score(X_train, y_train)
test_score = knn.score(X_test, y_test)

print('k: %d, Train Acc: %.3f, Test Acc: %.3f' % (k, train_score, test_score))

k: 5, Train Acc: 0.804, Test Acc: 0.680


In [689]:
predictions_2 = knn.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_2)))

Mean squared Error : 0.0494293297457


### Linear Regression:

In [690]:
# Linear Regression
from sklearn.linear_model import LinearRegression 

regressor = LinearRegression() 
regressor.fit(X_train, y_train) 
print ("Test score is: \n", regressor.score(X_test, y_test))

Test score is: 
 -345840.431275


In [691]:
predictions_3 = regressor.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_3)))

Mean squared Error : 53443.8980607


### Linear Regression with ridge regularization:

In [692]:
#to check what alpha value is better for the model

for i in range (-5, 5):
    alpha = 10**i
    rm = linear_model.Ridge(alpha=alpha)
    ridge_model = rm.fit(X_train, y_train)
    preds_ridge = ridge_model.predict(X_test)

    plt.scatter(preds_ridge, y_test, alpha=.75, color='b')
    plt.xlabel('Predicted Price')
    plt.ylabel('Actual Price')
    plt.title('Ridge Regularization with alpha = {}'.format(alpha))
    overlay = 'R^2 is: {}\nRMSE is: {}'.format(
                    ridge_model.score(X_test, y_test),
                    mean_squared_error(y_test, preds_ridge))
    plt.annotate(s=overlay,xy=(12.1,10.6),size='x-large')
    #plt.show()

In [693]:
from sklearn.metrics import mean_squared_error

linm = linear_model.Ridge(alpha = 10)

linm.fit(X_train, y_train)
print ("Test score is: \n", linm.score(X_test, y_test))

Test score is: 
 0.901239708666


In [694]:
predictions = linm.predict(X_test)
print("Mean Squared Error : " + str(mean_squared_error(y_test,predictions)))

Mean Squared Error : 0.0152617195777


### MLP with PCA:

In [695]:
#mlp_regressor = MLPRegressor(solver='lbfgs',hidden_layer_sizes = (200,5,5),alpha = 1.0, activation = 'relu', max_iter = 100)
#mlp_regressor.fit(X_train,y_train)

#print ("Test score is: \n", mlp_regressor.score(X_test, y_test))

In [696]:
#predictions7 = mlp_regressor.predict(X_test1)
#print("Mean Squared Error : " + str(mean_squared_error(y_test,predictions7)))

### Basic Decision tree:

In [697]:
# Define model
basic_decision_tree = DecisionTreeRegressor()

# Fit model
basic_decision_tree.fit(X, y)

print("R^2 is: \n", basic_decision_tree.score(X_test, y_test))

R^2 is: 
 0.999999996944


In [698]:
predictions_5 = basic_decision_tree.predict(X_test)
print("Mean Squared Error : " + str(mean_squared_error(y_test,predictions_5)))

Mean Squared Error : 4.72276373045e-10


### XGBoost

In [699]:
learning_rates = np.arange(0.0, 0.09, 0.0015)
learning_rates
best_learning_rates = [0.0885, 0.0735, 0.0705, 0.0615, 0.06, 0.0585, 0.057, 0.0555]

In [700]:
# to find best parameters
from xgboost import XGBRegressor

actual_values = y_test

for i in best_learning_rates:
    n_estimators = 1000
    my_model = XGBRegressor(n_estimators=n_estimators,learning_rate=i)
    xgboost_model = my_model.fit(X_train, y_train, early_stopping_rounds=5, 
                 eval_set=[(X_test, y_test)], verbose=False)
    preds_xgboost = xgboost_model.predict(X_test)

    plt.scatter(preds_xgboost, actual_values, alpha=.75, color='b')
    plt.xlabel('Predicted Price')
    plt.ylabel('Actual Price')
    plt.title('XGBoost with  = {}'.format(n_estimators))
    overlay = 'R^2 is: {}\nRMSE is: {}\nlearning Rate is: {}'.format(
                        xgboost_model.score(X_test, y_test),
                        mean_squared_error(y_test, preds_xgboost),
                        i)
    plt.annotate(s=overlay,xy=(12.1,10.6),size='x-large')
    #plt.show()

In [701]:
# XGBoost
XGBoost = XGBRegressor(n_estimators = 1000,learning_rate=0.0585)
XGBoost.fit(X_train, y_train)


print ("Test Score: \n", XGBoost.score(X_test, y_test))

Test Score: 
 0.905846405165


In [702]:
prediction = XGBoost.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,prediction)))

Mean squared Error : 0.0145498331587


### Lasso regression:

In [703]:
# Lasso regression
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
lasso.fit(X_train, y_train)
print ("Test Score: \n", lasso.score(X_test, y_test))

Test Score: 
 0.904965675904


In [704]:
predictions_lasso = lasso.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_lasso)))

Mean squared Error : 0.0146859348533


### Elastic Net Regression:

In [705]:
# Elastic Net regression
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
ENet.fit(X_train, y_train)
print ("Test Score: \n", ENet.score(X_test, y_test))

Test Score: 
 0.90495230902


In [706]:
predictions_ENet = ENet.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_ENet)))

Mean squared Error : 0.0146880004772


### Gradient Boosting Regression:

In [707]:
# Gradient boosting regression
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

GBoost.fit(X_train, y_train)
print ("Test Score: \n", GBoost.score(X_test, y_test))

Test Score: 
 0.900061935524


In [708]:
predictions_GBoost = GBoost.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_GBoost)))

Mean squared Error : 0.0154437243408


### Light GBM:

In [709]:
# LightGBM
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
model_lgb.fit(X_train, y_train)
print ("Test Score: \n", model_lgb.score(X_test, y_test))

Test Score: 
 0.902791193686


In [710]:
predictions_model_lgb = model_lgb.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_model_lgb)))

Mean squared Error : 0.0150219640142


### Avereging Models:

In [711]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone


class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

### Enet regression + GBoost + LGBM + Lasso regression (Best Score):

In [712]:
#Best version
averaged_models = AveragingModels(models = (ENet, GBoost, model_lgb, lasso))

averaged_models.fit(X_train, y_train)
print ("Test Score: \n", averaged_models.score(X_test, y_test))

Test Score: 
 0.910253536225


In [713]:
predictions = averaged_models.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions)))

Mean squared Error : 0.0138687861764


### Bagging method:

In [714]:
#from sklearn.ensemble import BaggingRegressor

#bagging = BaggingRegressor(KNeighborsClassifier(),max_samples=0.5, max_features=0.5)
#averaged_models.fit(X_train, y_train)
#print ("Test Score: \n", averaged_models.score(X_test, y_test))


In [715]:
#predictions = averaged_models.predict(X_test)
#print("Mean squared Error : " + str(mean_squared_error(y_test,predictions)))

### Stacking :
Using Meta model with cross validation

In [716]:



class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [717]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost),
                                                 meta_model = lasso)
stacked_averaged_models.fit(X_train, y_train)
print ("Test Score: \n", stacked_averaged_models.score(X_test, y_test))
predictions = stacked_averaged_models.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions)))

KeyError: '[  1   2   3   4   5   6   7   8   9  10  11  13  14  16  17  18  19  21\n  22  23  24  25  26  27  28  29  30  31  32  33  34  36  37  40  41  43\n  45  46  47  48  49  51  52  53  54  55  57  58  59  60  61  63  64  66\n  67  68  69  70  73  74  75  76  78  80  81  83  84  85  86  87  91  92\n  94  95  98  99 100 101 103 104 105 106 107 110 111 112 114 115 118 119\n 120 122 123 124 125 126 127 128 129 130 131 132 133 134 135 138 139 140\n 141 142 143 144 145 147 148 149 150 151 153 155 156 157 158 159 160 161\n 162 163 164 165 167 168 169 170 172 174 175 176 177 178 179 181 182 183\n 184 185 186 187 188 189 190 192 193 195 198 199 201 202 203 204 205 206\n 207 208 209 210 211 212 213 214 216 217 218 219 220 221 222 223 224 225\n 229 230 231 232 233 234 235 236 238 239 241 243 244 245 246 247 248 249\n 250 251 252 253 254 255 257 258 259 260 261 262 263 265 268 270 272 274\n 276 277 278 279 280 281 282 283 285 286 287 288 289 291 292 293 294 296\n 297 298 299 300 302 303 30

### Making a submission

In [ ]:
submission = pd.DataFrame()
%store -r test_ID
submission['Id'] = test_ID

In [ ]:
feats = test_set.select_dtypes(
        include=[np.number])
feats.shape

In [ ]:
predictions = stacked_averaged_models.predict(feats)
predictions
#final_predictions = np.exp(predictions)

#final_predictions = scalery.inverse_transform(predictions.reshape(-1,1))


In [ ]:
#submission['SalePrice'] = final_predictions
#submission.head(5)

In [ ]:
submission.to_csv("normalize_submission.csv", index=False)